In [1]:
import psutil
import platform

print("### Hardware Requirements ###")
print(f"CPU: {platform.processor()}")
print(f"Total RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

### Hardware Requirements ###
CPU: x86_64
Total RAM: 810.20 GB


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import platform

print("### Software Requirements ###")
print(f"Python Version: {platform.python_version()}")
print(f"TensorFlow Version: {tf.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")

2025-03-11 13:25:28.362901: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 13:25:29.707557: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Software Requirements ###
Python Version: 3.9.13
TensorFlow Version: 2.12.0
Pandas Version: 1.5.0
Numpy Version: 1.24.0


In [3]:
import time
start_time = time.time()

# BioMapAI Tutorial

This tutorial will guide you through:
1. **Environment Setup**  
2. **Data Preparation**  
3. **Model Initialization and Training**  
4. **Prediction and Evaluation**  
5. **Score Conversion and Weight Adjustment**  

By the end, you should be able to train your own model on custom data.


## 1. Environment Setup

1. **Download or Clone** the repository/project that contains:
   - **BioMapAI.py** (the script with model classes and methods)
   - **example_data/** (containing your `train_data.csv` and `test_data.csv`)
   - A script or notebook for your tutorial (this file)

2. **Install Required Libraries** (if you haven’t already):
```bash
pip install numpy pandas tensorflow
```

3. **Import modules** in Python:
```python
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import importlib.util
import os

# Clear any previous TensorFlow session
tf.keras.backend.clear_session()
```

4. **Function to import `BioMapAI.py`**:
```python
def import_module_with_full_path(file_path):
    base_filename = os.path.basename(file_path)
    module_name = os.path.splitext(base_filename)[0]
    module_spec = importlib.util.spec_from_file_location(module_name, file_path)
    imported_module = importlib.util.module_from_spec(module_spec)
    module_spec.loader.exec_module(imported_module)
    return imported_module
```


## 2. Data Preparation

Let’s assume you have:
- A **training dataset** named `train_data.csv`
- A **test dataset** named `test_data.csv`

Each row might look like:

| y0 (label) | Y1 | Y2 | ... | Y12 | X1 | X2 | ... | Xn |
|------------|----|----|-----|-----|----|----|-----|----|
| 0 or 1     | …  | …  | …   | …   | …  | …  | …   | …  |

Where:
- **y0** is your single-class label (e.g., 0 or 1)
- **Y** columns (Y1–Y12) are intermediate targets
- **X** columns (X1–Xn) are input features

### Load Your Data
```python
# 1) Import the BioMapAI module
BioMapAI = import_module_with_full_path("BioMapAI.py")

# 2) Load training data
data = pd.read_csv("example_data/train_data.csv", index_col=0)
y0 = data.iloc[:, 0]       # Single-class label
Y = data.iloc[:, 1:13]     # Intermediate target columns
X = data.iloc[:, 13:]      # Input features

# 3) Load test data
test_data = pd.read_csv("example_data/test_data.csv", index_col=0)
y0_test = test_data.iloc[:, 0]
Y_test = test_data.iloc[:, 1:13]
X_test = test_data.iloc[:, 13:]
```


## 3. Model Initialization and Training

### 3.1 **OmicScoreModel**
The **OmicScoreModel** is trained to predict the intermediate targets `Y`.

```python
model, model_history = BioMapAI.OmicScoreModel(
    epochs=500, 
    optimizer=tf.keras.optimizers.Adam(0.0005), 
    batch_size=64, 
    kernel_regularizer=tf.keras.regularizers.L2(0.008), 
    model_name="example"
).train(X, Y)
```
- **epochs**: Number of training iterations.
- **optimizer**: Choice of optimizer and learning rate.
- **batch_size**: Batch size for training.
- **kernel_regularizer**: Regularization to avoid overfitting.
- **model_name**: Useful if you want to save or track multiple models.

### 3.2 Monitoring the Training History
```python
print(model_history.history.keys())
# You can plot loss curves, accuracy, etc.
```


## 4. Prediction and Evaluation

### 4.1 Predict Intermediate Targets `Y`
```python
Y_pred = BioMapAI.OmicScoreModel().predict(model, X_test)
```
This outputs the intermediate predictions for each of the 12 `Y` columns.

### 4.2 Evaluate Intermediate Target Predictions
Compute the Mean Squared Error (MSE) on `Y_test`:
```python
mse = BioMapAI.OmicScoreModel().score(model, X_test, Y_test)
print("Intermediate Target MSE:", mse)
```


## 5. Score Conversion and Weight Adjustment

### 5.1 Build a Layer for Converting `Y` to `y0`
```python
Y_to_y_model = BioMapAI.ScoreLayer().build_model(Y, y0)
```
This creates a secondary model that maps the 12-dimensional `Y` to a single label or score.

### 5.2 Predict Final Labels `y0`
```python
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
```
### 5.3 Evaluate Final Model
```python
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Final Model Loss:", loss)
print("Final Model Accuracy:", accuracy)
```

### 5.4 Adjusting Weights (Optional)
If you want to fine-tune how the intermediate predictions map to the final label:
```python
Y_to_y_model_adjust = BioMapAI.WeightsAdjust(model, X, Y, Y_to_y_model).adjust_score_weight()
```
Then re-predict and re-evaluate:
```python
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Adjusted Final Model Loss:", loss)
print("Adjusted Final Model Accuracy:", accuracy)
```


## Putting It All Together

Below is a compact version of the entire flow. You can copy this code into its own file (e.g., `train_and_eval.py`) or run it as a Jupyter cell.


In [4]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import random
import tensorflow as tf
tf.keras.backend.clear_session()

import importlib.util
import os

def import_module_with_full_path(file_path):
    base_filename = os.path.basename(file_path)
    module_name = os.path.splitext(base_filename)[0]
    module_spec = importlib.util.spec_from_file_location(module_name, file_path)
    imported_module = importlib.util.module_from_spec(module_spec)
    module_spec.loader.exec_module(imported_module)
    return imported_module

# 1) Import BioMapAI module
BioMapAI = import_module_with_full_path("BioMapAI.py")

# 2) Load data
data = pd.read_csv("example_data/train_data.csv", index_col=0)
y0 = data.iloc[:, 0]
Y = data.iloc[:, 1:13]
X = data.iloc[:, 13:]

test_data = pd.read_csv("example_data/test_data.csv", index_col=0)
y0_test = test_data.iloc[:, 0]
Y_test = test_data.iloc[:, 1:13]
X_test = test_data.iloc[:, 13:]

# 3) Train OmicScoreModel
model, model_history = BioMapAI.OmicScoreModel(
    epochs=500,
    optimizer=tf.keras.optimizers.Adam(0.0005),
    batch_size=64,
    kernel_regularizer=tf.keras.regularizers.L2(0.008),
    model_name="example"
).train(X, Y)

# 4) Predict and evaluate intermediate targets
Y_pred = BioMapAI.OmicScoreModel().predict(model, X_test)
mse = BioMapAI.OmicScoreModel().score(model, X_test, Y_test)
print("Intermediate Target MSE:", mse)

# 5) Build layer to convert Y --> y0
Y_to_y_model = BioMapAI.ScoreLayer().build_model(Y, y0)

# 6) Predict final output
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)

# 7) Evaluate final output
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Initial Final Model Loss:", loss)
print("Initial Final Model Accuracy:", accuracy)

# 8) (Optional) Adjust weights
Y_to_y_model_adjust = BioMapAI.WeightsAdjust(model, X, Y, Y_to_y_model).adjust_score_weight()

# 9) Predict & Evaluate again
y_pred = BioMapAI.ScoreYModel(model, Y_to_y_model).predict(X_test)
loss, accuracy = BioMapAI.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Adjusted Final Model Loss:", loss)
print("Adjusted Final Model Accuracy:", accuracy)

Epoch 1/500
3/3 [==============================] - 6s 13ms/step - loss: 8.4081 - out0_loss: 0.1109 - out1_loss: 2.2387 - out2_loss: 1.7021 - out3_loss: 0.2325 - out4_loss: 0.3025 - out5_loss: 0.1470 - out6_loss: 0.0774 - out7_loss: 0.4811 - out8_loss: 0.2541 - out9_loss: 0.3736 - out10_loss: 0.1076 - out11_loss: 0.1441 - out0_mse: 0.2218 - out1_accuracy: 0.1714 - out2_accuracy: 0.3200 - out3_mse: 0.4653 - out4_mse: 0.7085 - out5_mse: 0.2961 - out6_mse: 0.1567 - out7_mse: 1.5634 - out8_mse: 0.6303 - out9_mse: 0.9767 - out10_mse: 0.2153 - out11_mse: 0.3113
Epoch 2/500
3/3 [==============================] - 0s 8ms/step - loss: 8.1666 - out0_loss: 0.1062 - out1_loss: 2.0096 - out2_loss: 1.6381 - out3_loss: 0.2505 - out4_loss: 0.3165 - out5_loss: 0.1167 - out6_loss: 0.0742 - out7_loss: 0.5093 - out8_loss: 0.2823 - out9_loss: 0.3908 - out10_loss: 0.1213 - out11_loss: 0.1235 - out0_mse: 0.2124 - out1_accuracy: 0.1829 - out2_accuracy: 0.2914 - out3_mse: 0.5011 - out4_mse: 0.8463 - out5_mse: 0.

3/3 [==============================] - 0s 8ms/step - loss: 7.1435 - out0_loss: 0.1080 - out1_loss: 1.8549 - out2_loss: 1.4402 - out3_loss: 0.2197 - out4_loss: 0.1943 - out5_loss: 0.1221 - out6_loss: 0.0762 - out7_loss: 0.3406 - out8_loss: 0.1808 - out9_loss: 0.2443 - out10_loss: 0.1267 - out11_loss: 0.1105 - out0_mse: 0.2160 - out1_accuracy: 0.2914 - out2_accuracy: 0.3314 - out3_mse: 0.4411 - out4_mse: 0.4481 - out5_mse: 0.2443 - out6_mse: 0.1525 - out7_mse: 0.9650 - out8_mse: 0.4276 - out9_mse: 0.5923 - out10_mse: 0.2546 - out11_mse: 0.2386
Epoch 16/500
3/3 [==============================] - 0s 8ms/step - loss: 7.0800 - out0_loss: 0.1075 - out1_loss: 1.8138 - out2_loss: 1.4048 - out3_loss: 0.2086 - out4_loss: 0.2128 - out5_loss: 0.1171 - out6_loss: 0.0747 - out7_loss: 0.3662 - out8_loss: 0.1749 - out9_loss: 0.2589 - out10_loss: 0.1104 - out11_loss: 0.1124 - out0_mse: 0.2151 - out1_accuracy: 0.2286 - out2_accuracy: 0.4000 - out3_mse: 0.4220 - out4_mse: 0.4761 - out5_mse: 0.2341 - out6_

3/3 [==============================] - 0s 8ms/step - loss: 6.6027 - out0_loss: 0.1037 - out1_loss: 1.7492 - out2_loss: 1.4558 - out3_loss: 0.2126 - out4_loss: 0.1367 - out5_loss: 0.1062 - out6_loss: 0.0795 - out7_loss: 0.2529 - out8_loss: 0.1163 - out9_loss: 0.1998 - out10_loss: 0.1038 - out11_loss: 0.0611 - out0_mse: 0.2075 - out1_accuracy: 0.2857 - out2_accuracy: 0.3657 - out3_mse: 0.4255 - out4_mse: 0.2871 - out5_mse: 0.2125 - out6_mse: 0.1625 - out7_mse: 0.6308 - out8_mse: 0.2407 - out9_mse: 0.4297 - out10_mse: 0.2077 - out11_mse: 0.1231
Epoch 30/500
3/3 [==============================] - 0s 8ms/step - loss: 6.4312 - out0_loss: 0.1019 - out1_loss: 1.7038 - out2_loss: 1.3297 - out3_loss: 0.2033 - out4_loss: 0.1254 - out5_loss: 0.1042 - out6_loss: 0.0753 - out7_loss: 0.2419 - out8_loss: 0.1463 - out9_loss: 0.2152 - out10_loss: 0.1009 - out11_loss: 0.0654 - out0_mse: 0.2038 - out1_accuracy: 0.3143 - out2_accuracy: 0.4114 - out3_mse: 0.4065 - out4_mse: 0.2596 - out5_mse: 0.2084 - out6_

3/3 [==============================] - 0s 7ms/step - loss: 5.9387 - out0_loss: 0.1035 - out1_loss: 1.5032 - out2_loss: 1.2669 - out3_loss: 0.1747 - out4_loss: 0.1007 - out5_loss: 0.1202 - out6_loss: 0.0734 - out7_loss: 0.2022 - out8_loss: 0.1165 - out9_loss: 0.1818 - out10_loss: 0.1096 - out11_loss: 0.0542 - out0_mse: 0.2071 - out1_accuracy: 0.3829 - out2_accuracy: 0.4000 - out3_mse: 0.3495 - out4_mse: 0.2056 - out5_mse: 0.2413 - out6_mse: 0.1469 - out7_mse: 0.4614 - out8_mse: 0.2463 - out9_mse: 0.3735 - out10_mse: 0.2193 - out11_mse: 0.1102
Epoch 44/500
3/3 [==============================] - 0s 8ms/step - loss: 5.9353 - out0_loss: 0.1025 - out1_loss: 1.5349 - out2_loss: 1.2564 - out3_loss: 0.1904 - out4_loss: 0.0788 - out5_loss: 0.1119 - out6_loss: 0.0739 - out7_loss: 0.2118 - out8_loss: 0.1151 - out9_loss: 0.1786 - out10_loss: 0.1034 - out11_loss: 0.0524 - out0_mse: 0.2049 - out1_accuracy: 0.3943 - out2_accuracy: 0.5257 - out3_mse: 0.3808 - out4_mse: 0.1592 - out5_mse: 0.2238 - out6_

3/3 [==============================] - 0s 8ms/step - loss: 5.5209 - out0_loss: 0.0945 - out1_loss: 1.4053 - out2_loss: 1.1911 - out3_loss: 0.1750 - out4_loss: 0.0580 - out5_loss: 0.1035 - out6_loss: 0.0781 - out7_loss: 0.1633 - out8_loss: 0.0906 - out9_loss: 0.1639 - out10_loss: 0.0982 - out11_loss: 0.0541 - out0_mse: 0.1890 - out1_accuracy: 0.4971 - out2_accuracy: 0.5314 - out3_mse: 0.3562 - out4_mse: 0.1161 - out5_mse: 0.2069 - out6_mse: 0.1562 - out7_mse: 0.3384 - out8_mse: 0.1839 - out9_mse: 0.3286 - out10_mse: 0.1965 - out11_mse: 0.1093
Epoch 58/500
3/3 [==============================] - 0s 9ms/step - loss: 5.5825 - out0_loss: 0.0988 - out1_loss: 1.4384 - out2_loss: 1.2436 - out3_loss: 0.1580 - out4_loss: 0.0575 - out5_loss: 0.0951 - out6_loss: 0.0678 - out7_loss: 0.1898 - out8_loss: 0.0872 - out9_loss: 0.1668 - out10_loss: 0.0933 - out11_loss: 0.0469 - out0_mse: 0.1975 - out1_accuracy: 0.4343 - out2_accuracy: 0.4686 - out3_mse: 0.3160 - out4_mse: 0.1152 - out5_mse: 0.1902 - out6_

...

3/3 [==============================] - 0s 8ms/step - loss: 1.5612 - out0_loss: 0.0180 - out1_loss: 0.2725 - out2_loss: 0.5392 - out3_loss: 0.0229 - out4_loss: 0.0133 - out5_loss: 0.0205 - out6_loss: 0.0160 - out7_loss: 0.0193 - out8_loss: 0.0120 - out9_loss: 0.0136 - out10_loss: 0.0158 - out11_loss: 0.0163 - out0_mse: 0.0360 - out1_accuracy: 0.9086 - out2_accuracy: 0.8114 - out3_mse: 0.0459 - out4_mse: 0.0265 - out5_mse: 0.0409 - out6_mse: 0.0320 - out7_mse: 0.0385 - out8_mse: 0.0240 - out9_mse: 0.0272 - out10_mse: 0.0316 - out11_mse: 0.0326
Epoch 478/500
3/3 [==============================] - 0s 8ms/step - loss: 1.5721 - out0_loss: 0.0192 - out1_loss: 0.2759 - out2_loss: 0.5398 - out3_loss: 0.0244 - out4_loss: 0.0131 - out5_loss: 0.0179 - out6_loss: 0.0196 - out7_loss: 0.0220 - out8_loss: 0.0124 - out9_loss: 0.0132 - out10_loss: 0.0172 - out11_loss: 0.0172 - out0_mse: 0.0384 - out1_accuracy: 0.9029 - out2_accuracy: 0.8057 - out3_mse: 0.0488 - out4_mse: 0.0262 - out5_mse: 0.0357 - out6

3/3 [==============================] - 0s 8ms/step - loss: 1.5055 - out0_loss: 0.0176 - out1_loss: 0.2745 - out2_loss: 0.5004 - out3_loss: 0.0255 - out4_loss: 0.0112 - out5_loss: 0.0198 - out6_loss: 0.0167 - out7_loss: 0.0207 - out8_loss: 0.0121 - out9_loss: 0.0132 - out10_loss: 0.0152 - out11_loss: 0.0162 - out0_mse: 0.0352 - out1_accuracy: 0.9029 - out2_accuracy: 0.8000 - out3_mse: 0.0509 - out4_mse: 0.0224 - out5_mse: 0.0397 - out6_mse: 0.0335 - out7_mse: 0.0414 - out8_mse: 0.0243 - out9_mse: 0.0264 - out10_mse: 0.0305 - out11_mse: 0.0323
Epoch 492/500
3/3 [==============================] - 0s 8ms/step - loss: 1.5529 - out0_loss: 0.0193 - out1_loss: 0.3135 - out2_loss: 0.4956 - out3_loss: 0.0235 - out4_loss: 0.0111 - out5_loss: 0.0204 - out6_loss: 0.0186 - out7_loss: 0.0255 - out8_loss: 0.0131 - out9_loss: 0.0157 - out10_loss: 0.0183 - out11_loss: 0.0173 - out0_mse: 0.0387 - out1_accuracy: 0.8914 - out2_accuracy: 0.7943 - out3_mse: 0.0470 - out4_mse: 0.0222 - out5_mse: 0.0407 - out6

6/6 [==============================] - 0s 1ms/step - loss: 0.1586 - accuracy: 0.9771
Epoch 24/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1541 - accuracy: 0.9771
Epoch 25/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1500 - accuracy: 0.9771
Epoch 26/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1463 - accuracy: 0.9771
Epoch 27/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1428 - accuracy: 0.9771
Epoch 28/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1405 - accuracy: 0.9771
Epoch 29/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1367 - accuracy: 0.9886
Epoch 30/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1343 - accuracy: 0.9829
Epoch 31/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1320 - accuracy: 0.9829
Epoch 32/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1296 - accuracy: 0.9886
Epoch 33/200
6/6 [===============

6/6 [==============================] - 0s 1ms/step - loss: 0.0854 - accuracy: 0.9829
Epoch 107/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0855 - accuracy: 0.9829
Epoch 108/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0853 - accuracy: 0.9829
Epoch 109/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0852 - accuracy: 0.9829
Epoch 110/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0848 - accuracy: 0.9829
Epoch 111/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0849 - accuracy: 0.9829
Epoch 112/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0850 - accuracy: 0.9829
Epoch 113/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0846 - accuracy: 0.9829
Epoch 114/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0844 - accuracy: 0.9829
Epoch 115/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0844 - accuracy: 0.9829
Epoch 116/200
6/6 [=====

6/6 [==============================] - 0s 1ms/step - loss: 0.0782 - accuracy: 0.9829
Epoch 189/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0780 - accuracy: 0.9829
Epoch 190/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0780 - accuracy: 0.9829
Epoch 191/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0780 - accuracy: 0.9829
Epoch 192/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0779 - accuracy: 0.9829
Epoch 193/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0780 - accuracy: 0.9829
Epoch 194/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0778 - accuracy: 0.9829
Epoch 195/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0777 - accuracy: 0.9829
Epoch 196/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0777 - accuracy: 0.9829
Epoch 197/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0777 - accuracy: 0.9829
Epoch 198/200
6/6 [=====

## Conclusion
You have now:
1. **Loaded and prepared** your data
2. **Trained** the OmicScoreModel on intermediate targets
3. **Transformed** those intermediate predictions to final labels
4. **Evaluated** and **(optionally) adjusted** model weights

Feel free to experiment with different architectures, hyperparameters (learning rate, batch size, etc.), or data splits. Modify **BioMapAI.py** to suit your specific needs.

**Happy modeling!**

In [5]:
end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")

Total execution time: 23.61 seconds
